# Consultas MONGODB

Nicolás Fernández y Néstor Villa

In [1]:
# Importamos las clases y el método init_app
from models import Cliente, Producto, Compra, Proveedor, init_app
from bson import ObjectId
from datetime import datetime

# Iniciamos la conexión a la base de datos
init_app()

In [2]:
#1. Listado de todas las compras de un cliente
pipeline = [
    {
        "$group": {
            "_id": "$cliente.nombre",
            "compras": { "$push": { "$toString": "$_id" } }
        }
    },
    {
        "$project": {
            "nombre_cliente": "$_id",
            "compras": 1,
            "_id": 0
        }
    }
]

# Ejecuta el pipeline con raw=True
compras_cliente = list(Compra.aggregate(pipeline, raw=True))

compras_cliente

[{'compras': ['671ea82a13d6f9c062f6b5e5',
   '671ea82a13d6f9c062f6b5e7',
   '671ea82b13d6f9c062f6b5ed',
   '671ea82b13d6f9c062f6b5ef'],
  'nombre_cliente': 'Luis Martínez'},
 {'compras': ['671ea82813d6f9c062f6b5da',
   '671ea82813d6f9c062f6b5db',
   '671ea82913d6f9c062f6b5dc',
   '671ea82913d6f9c062f6b5dd',
   '671ea82b13d6f9c062f6b5f0',
   '671ea82b13d6f9c062f6b5f3',
   '671ea82b13d6f9c062f6b5f4',
   '671ea82b13d6f9c062f6b5f5',
   '671ea82b13d6f9c062f6b5f6',
   '671ea82b13d6f9c062f6b5f7'],
  'nombre_cliente': 'Beatriz Gómez'},
 {'compras': ['671ea82913d6f9c062f6b5df',
   '671ea82913d6f9c062f6b5e0',
   '671ea82913d6f9c062f6b5e1',
   '671ea82913d6f9c062f6b5e2',
   '671ea82a13d6f9c062f6b5e3',
   '671ea82b13d6f9c062f6b5e8',
   '671ea82b13d6f9c062f6b5ec',
   '671ea82b13d6f9c062f6b5f2'],
  'nombre_cliente': 'María Fernández'},
 {'compras': ['671ea82a13d6f9c062f6b5e4',
   '671ea82a13d6f9c062f6b5e6',
   '671ea82b13d6f9c062f6b5e9',
   '671ea82b13d6f9c062f6b5ea',
   '671ea82b13d6f9c062f6b5eb',


In [3]:
# 2. Listado de todos los proveedores para un producto
pipeline = [
    { "$unwind": "$proveedores" },
    {
        "$group": {
            "_id": "$nombre",
            "proveedores": { "$push": "$proveedores.nombre" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "nombre_producto": "$_id",
            "proveedores": 1
        }
    }
]

# Ejecuta el pipeline con raw=True
proveedores_producto = list(Producto.aggregate(pipeline, raw=True))

proveedores_producto

[{'proveedores': ['Distribuciones Norte'],
  'nombre_producto': 'Pantalón vaquero'},
 {'proveedores': ['Proveedor Central', 'Logística Sur'],
  'nombre_producto': 'Zapatos deportivos'},
 {'proveedores': ['Modas Paqui'],
  'nombre_producto': 'Camiseta de manga corta roja'},
 {'proveedores': ['Modas Paqui', 'Logística Sur', 'Proveedor Central'],
  'nombre_producto': 'Camiseta de manga corta azul'},
 {'proveedores': ['Modas Paqui', 'Proveedor Central', 'Logística Sur'],
  'nombre_producto': 'Polo de manga corta verde'},
 {'proveedores': ['Proveedor Central'], 'nombre_producto': 'Sombrero de paja'},
 {'proveedores': ['Distribuciones Norte', 'Logística Sur', 'Modas Paqui'],
  'nombre_producto': 'Traje de baño'},
 {'proveedores': ['Proveedor Este', 'Distribuciones Norte', 'Modas Paqui'],
  'nombre_producto': 'Jersey de lana'},
 {'proveedores': ['Modas Paqui', 'Proveedor Este'],
  'nombre_producto': 'Camiseta de manga corta blanca'},
 {'proveedores': ['Proveedor Este'],
  'nombre_producto': '

In [4]:
# 3. Listado de todos los productos diferentes comprados por un cliente
pipeline = [
    { "$unwind": "$productos" },
    {
        "$group": {
            "_id": "$cliente.nombre",
            "productos": { "$addToSet": "$productos.nombre" }
        }
    },
    {
        "$project": {
            "nombre_cliente": "$_id",
            "productos": 1,
            "_id": 0
        }
    }
]

# Ejecuta el pipeline con raw=True
productos_cliente = list(Compra.aggregate(pipeline, raw=True))

# Muestra los resultados
productos_cliente    

[{'productos': ['Camiseta de manga corta blanca',
   'Falda plisada',
   'Vestido de fiesta',
   'Gafas de sol',
   'Guantes de invierno'],
  'nombre_cliente': 'Ana Sánchez'},
 {'productos': ['Bufanda de seda',
   'Zapatos de vestir',
   'Vestido de fiesta',
   'Chaqueta de cuero',
   'Camisa de manga larga',
   'Guantes de invierno',
   'Calcetines de algodón',
   'Camiseta de manga corta azul',
   'Zapatos deportivos',
   'Camiseta de manga corta roja',
   'Sombrero de paja',
   'Pantalón vaquero',
   'Jersey de lana',
   'Polo de manga corta verde',
   'Traje de baño',
   'Cinturón de cuero',
   'Camiseta de manga corta blanca',
   'Camiseta de manga corta negra',
   'Falda plisada'],
  'nombre_cliente': 'Beatriz Gómez'},
 {'productos': ['Bufanda de seda',
   'Vestido de fiesta',
   'Gafas de sol',
   'Chaqueta de cuero',
   'Guantes de invierno',
   'Calcetines de algodón',
   'Pantalón vaquero',
   'Zapatos deportivos',
   'Camiseta de manga corta roja',
   'Camiseta de manga cort

In [5]:
# 4. Listado de productos vendidos por “Modas Paqui” cuyo nombre contenga “manga corta”
pipeline = [
    { "$unwind": "$proveedores" },
    {
        "$match": {
            "proveedores.nombre": "Modas Paqui",
            "nombre": { "$regex": "manga corta", "$options": "i" }
        }
    },
    {
        "$project": {
            "_id": 0,
            "nombre_producto": "$nombre",
            "proveedor": "$proveedores.nombre"
        }
    }
]

# Ejecuta el pipeline con raw=True
productos_manga_corta = list(Producto.aggregate(pipeline, raw=True))

productos_manga_corta

[{'nombre_producto': 'Camiseta de manga corta azul',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Camiseta de manga corta blanca',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Camiseta de manga corta negra',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Camiseta de manga corta roja',
  'proveedor': 'Modas Paqui'},
 {'nombre_producto': 'Polo de manga corta verde', 'proveedor': 'Modas Paqui'}]

In [32]:
# 5. Calcular el peso y volumen total de los productos comprados por un cliente un día determinado
cliente_nombre = "María Fernández"  # Reemplaza con el nombre real del cliente
fecha_especifica = datetime(2023, 11, 4)  # Reemplaza con la fecha específica

pipeline = [
    # Filtrar las compras por cliente y fecha
    {
        "$match": {
            "cliente.nombre": cliente_nombre,
            "fecha_compra": fecha_especifica
        }
    },
    # Descomponer el arreglo de productos
    {"$unwind": "$productos"},
    # Calcular el volumen de cada producto
    {
        "$addFields": {
            "productos.volumen": {
                "$multiply": [
                    "$productos.dimensiones.ancho",
                    "$productos.dimensiones.alto",
                    "$productos.dimensiones.profundidad"
                ]
            }
        }
    },
    # Sumar el peso y el volumen de todos los productos
    {
        "$group": {
            "_id": None,
            "peso_total": { "$sum": "$productos.peso" },
            "volumen_total": { "$sum": "$productos.volumen" }
        }
    },
    {"$project": {"_id": 0,}}
]


# Ejecuta el pipeline con raw=True
peso_volumen_cliente = list(Compra.aggregate(pipeline, raw=True))

peso_volumen_cliente

[{'peso_total': 18.52, 'volumen_total': 50868}]

In [36]:
# 6. Calcular el número medio de envíos por mes y almacén
pipeline = [
    # Descomponer el arreglo de productos
    {"$unwind": "$productos"},
    # Descomponer el arreglo de proveedores por producto
    {"$unwind": "$productos.proveedores"},
    # Descomponer el arreglo de direcciones de almacenes por proveedor
    {"$unwind": "$productos.proveedores.direcciones_almacenes"},
    # Extraer el mes y el año de la fecha de compra
    {
        "$addFields": {
            "mes": { "$month": "$fecha_compra" },
            "anio": { "$year": "$fecha_compra" },
            "almacen": "$productos.proveedores.direcciones_almacenes"
        }
    },
    # Agrupar por almacén, mes y año para contar los envíos
    {
        "$group": {
            "_id": {
                "almacen": "$almacen",
                "mes": "$mes",
                "anio": "$anio"
            },
            "envios": { "$sum": 1 }
        }
    },
    # Calcular el promedio de envíos por mes y almacén
    {
        "$group": {
            "_id": {
                "almacen": "$_id.almacen",
                "mes": "$_id.mes"
            },
            "media_envios": { "$avg": "$envios" }
        }
    },
    # Agrupar nuevamente por almacén para recopilar los meses y medias
    {
        "$group": {
            "_id": "$_id.almacen",
            "envios_por_mes": {
                "$push": {
                    "mes": "$_id.mes",
                    "media_envios": "$media_envios"
                }
            }
        }
    },
    # Ordenar los meses dentro de cada almacén
    {
        "$project": {
            "_id": 0,
            "almacen": "$_id",
            "envios_por_mes": {
                "$sortArray": {
                    "input": "$envios_por_mes",
                    "sortBy": { "mes": 1 }
                }
            }
        }
    },
    # Ordenar los almacenes por ciudad para mejor legibilidad (opcional)
    {
        "$sort": {
            "almacen.ciudad": 1
        }
    }
]

# Ejecuta el pipeline con raw=True
promedio_envios = list(Compra.aggregate(pipeline, raw=True))

promedio_envios

[{'almacen': {'calle': 'Rua Augusta',
   'numero': '100',
   'ciudad': 'Lisboa',
   'codigo_postal': '1100-053',
   'pais': 'Portugal',
   'location': {'type': 'Point', 'coordinates': [-9.137398, 38.710141]}},
  'envios_por_mes': [{'mes': 1, 'media_envios': 2.0},
   {'mes': 2, 'media_envios': 12.0},
   {'mes': 3, 'media_envios': 2.0},
   {'mes': 4, 'media_envios': 6.0},
   {'mes': 5, 'media_envios': 2.0},
   {'mes': 7, 'media_envios': 4.0},
   {'mes': 8, 'media_envios': 1.0},
   {'mes': 9, 'media_envios': 4.0},
   {'mes': 11, 'media_envios': 3.0},
   {'mes': 12, 'media_envios': 4.0}]},
 {'almacen': {'calle': 'Calle de la Moda',
   'numero': '123',
   'ciudad': 'Madrid',
   'codigo_postal': '28015',
   'pais': 'España',
   'location': {'type': 'Point', 'coordinates': [-3.70379, 40.416775]}},
  'envios_por_mes': [{'mes': 1, 'media_envios': 3.0},
   {'mes': 2, 'media_envios': 9.0},
   {'mes': 3, 'media_envios': 5.0},
   {'mes': 4, 'media_envios': 10.0},
   {'mes': 5, 'media_envios': 4.0},

In [34]:
# 7. Listado con los tres proveedores con más volumen de facturación
pipeline = [
    # Descomponer el arreglo de productos
    {"$unwind": "$productos"},
    # Descomponer el arreglo de proveedores por producto
    {"$unwind": "$productos.proveedores"},
    # Agrupar por proveedor y sumar el precio de los productos
    {
        "$group": {
            "_id": "$productos.proveedores.nombre",
            "total_facturacion": { "$sum": "$productos.precio" }
        }
    },
    # Ordenar por total_facturacion descendente
    {
        "$sort": { "total_facturacion": -1 }
    },
    # Limitar a los tres primeros
    {
        "$limit": 3
    }
]


# Ejecuta el pipeline con raw=True
top_proveedores = list(Compra.aggregate(pipeline, raw=True))

top_proveedores

[{'_id': 'Logística Sur', 'total_facturacion': 5839.69},
 {'_id': 'Proveedor Este', 'total_facturacion': 5414.24},
 {'_id': 'Modas Paqui', 'total_facturacion': 5044.45}]

In [16]:
# 8. Listado de almacenes cerca de unas coordenadas determinadas (100km de distancia máxima)
pipeline = [
    {
        "$geoNear": {
            "near": { "type": "Point", "coordinates": [-9.537398, 38.710141] },
            "distanceField": "distancia",
            "maxDistance": 100 * 1000,
            "spherical": True,
            "query": { "direcciones_almacenes.location": { "$exists": True } }
        }
    },
    { "$unwind": "$direcciones_almacenes" },
    { "$sort": { "distancia": 1 } },
    {
        "$project": {
            "_id": 0,
            "calle_almacen": "$direcciones_almacenes.calle",
            "ciudad": "$direcciones_almacenes.ciudad",
            "distancia": 1,
            "location": "$direcciones_almacenes.location"
        }
    }
]

# Ejecuta el pipeline con raw=True
almacenes_cercanos = list(Proveedor.aggregate(pipeline, raw=True))

almacenes_cercanos

[{'distancia': 34745.68888511416,
  'calle_almacen': 'Rua Augusta',
  'ciudad': 'Lisboa',
  'location': {'type': 'Point', 'coordinates': [-9.137398, 38.710141]}}]

In [20]:
# 9. Listado de compras dentro de un polígono cuyos vértices están definidos
# Definir los vértices del polígono (longitud, latitud)
poligono = {
    "type": "Polygon",
    "coordinates": [[
        [-3.8880, 40.5687],
        [-3.5499, 40.5687],
        [-3.5499, 40.3120],
        [-3.8880, 40.3120],
        [-3.8880, 40.5687] # El polígono debe ser cerrado (el primer y último punto deben ser iguales)
    ]]
}

pipeline = [
    {
        "$match": {
            "direccion_envio.location": {
                "$geoWithin": {
                    "$geometry": poligono
                }
            }
        }
    },
    # Opcional: proyectar campos específicos
    {
        "$project": {
            "cliente.nombre": 1,
            "fecha_compra": 1,
            "direccion_envio": 1,
            "_id": 0
        }
    }
]


# Ejecuta el pipeline con raw=True
compras_poligono = list(Compra.aggregate(pipeline, raw=True))

compras_poligono

[{'cliente': {'nombre': 'Luis Martínez'},
  'fecha_compra': datetime.datetime(2023, 11, 4, 0, 0),
  'direccion_envio': {'calle': 'Calle Mayor',
   'ciudad': 'Madrid',
   'codigo_postal': '28013',
   'pais': 'España',
   'numero': '63',
   'location': {'type': 'Point', 'coordinates': [-3.709973, 40.415571]}}},
 {'cliente': {'nombre': 'Luis Martínez'},
  'fecha_compra': datetime.datetime(2024, 9, 18, 0, 0),
  'direccion_envio': {'calle': 'Calle Mayor',
   'ciudad': 'Madrid',
   'codigo_postal': '28013',
   'pais': 'España',
   'numero': '63',
   'location': {'type': 'Point', 'coordinates': [-3.709973, 40.415571]}}},
 {'cliente': {'nombre': 'Luis Martínez'},
  'fecha_compra': datetime.datetime(2024, 9, 5, 0, 0),
  'direccion_envio': {'calle': 'Calle Mayor',
   'ciudad': 'Madrid',
   'codigo_postal': '28013',
   'pais': 'España',
   'numero': '63',
   'location': {'type': 'Point', 'coordinates': [-3.709973, 40.415571]}}},
 {'cliente': {'nombre': 'Luis Martínez'},
  'fecha_compra': datetime

In [16]:
# 10. Guardar el listado de compras en una nueva colección
pipeline = [
    {
        "$match": {
            "fecha_compra": datetime(2024, 4, 11)
        }
    },
    { "$unwind": "$productos" },
    { "$unwind": "$productos.proveedores" },
    { "$unwind": "$productos.proveedores.direcciones_almacenes" },
    {
        "$match": {
            "productos.proveedores.direcciones_almacenes.calle": "Avenida de Andalucía",
            "productos.proveedores.direcciones_almacenes.ciudad": "Sevilla",
            "productos.proveedores.direcciones_almacenes.location.coordinates": [-5.963709, 37.387533]
        }
    },
    {
        "$project": {
            "_id": 1,
            "cliente": "$cliente.nombre",
            "productos": "$productos.nombre",
            "precio_compra": 1,
            "fecha_compra": 1,
            "direccion_envio": 1,
            "almacen": {
                "direccion": "$productos.proveedores.direcciones_almacenes.calle",
                "ciudad": "$productos.proveedores.direcciones_almacenes.ciudad",
                "location": "$productos.proveedores.direcciones_almacenes.location"
            }
        }
    },
    {
        "$out": "compras_a_enviar"
    }
]

Compra.aggregate(pipeline)

compras_a_enviar = list(Compra.db.database["compras_a_enviar"].find())
compras_a_enviar

[{'_id': ObjectId('671e87d36018dfcfed045212'),
  'precio_compra': 158.92000000000002,
  'fecha_compra': datetime.datetime(2024, 4, 11, 0, 0),
  'direccion_envio': None,
  'cliente': 'Beatriz Gómez',
  'productos': 'Gafas de sol',
  'almacen': {'direccion': 'Avenida de Andalucía',
   'ciudad': 'Sevilla',
   'location': {'type': 'Point', 'coordinates': [-5.963709, 37.387533]}}},
 {'_id': ObjectId('671e87d36018dfcfed045213'),
  'precio_compra': 471.22,
  'fecha_compra': datetime.datetime(2024, 4, 11, 0, 0),
  'direccion_envio': None,
  'cliente': 'Beatriz Gómez',
  'productos': 'Traje de baño',
  'almacen': {'direccion': 'Avenida de Andalucía',
   'ciudad': 'Sevilla',
   'location': {'type': 'Point', 'coordinates': [-5.963709, 37.387533]}}}]